# Extract thông tin tài chính từ tin tức dùng VectorStoreIndex base

Để đánh giá tình hình tài chính của một công ty, một trong những nguồn thông tin quan trọng là các bài báo, tin tức về công ty đó. Tuy nhiên, việc đọc và tổng hợp thông tin từ các bài báo là một công việc tốn thời gian và công sức. Trong bài toán này, chúng ta sẽ xây dựng một mô hình để tự động trích xuất thông tin tài chính từ các bài báo về công ty.

```
{công ty A} => {có_lợi_nhuận, không_có_lợi_nhuận, lợi_nhuận_tăng, lợi_nhuận_giảm, v.v.} => {tiền_tệ, số_tiền, v.v.}
```

https://neo4j.com/labs/genai-ecosystem/llamaindex/


In [1]:
from dotenv import load_dotenv
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

import nest_asyncio
nest_asyncio.apply()
# Install the python-dotenv package if not already installed
%pip install python-dotenv

# Load the .env file

load_dotenv('../.env')

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(model="gpt-4o-mini", api_key=openai_key)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

FOLDER_DATA = "../data/bao-chi"

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

documents = SimpleDirectoryReader(FOLDER_DATA).load_data()
print("length of documents:", len(documents))
print('documents:', documents[1])



length of documents: 24
documents: Doc ID: 420af5a2-8901-49ad-b40a-4a9b26fd2db4
Text: Khối ngoại bất ngờ có tuần bán ròng hơn 2.000 tỷ, cổ phiếu “đại
gia” công nghệ số 1 Việt Nam dẫn đầu danh sách xả hàng


In [4]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(documents, embed_model)
query_engine = vector_index.as_query_engine()

In [5]:
test_pairs = [
    ("lãi sau thuế của Chứng khoán LPBank ?", "Tăng 275% so với cùng kỳ năm trước"),
    ("thu gom cố phiếu MSN?", "Khối ngoại cũng giải ngân mua ròng 179 tỷ tại cổ phiếu MSN"),
    ("Doanh thu Bia Sài Gòn - Quảng Ngãi (BSQ) trong BCTC quý 3/2024 ?", "Bia Sài Gòn - Quảng Ngãi (BSQ) ghi nhận doanh thu 418 tỷ đồng"),
    ("Lực mua ròng các cổ phiếu nào ?", "ACV', 'YEG', 'TCB', 'TPB', 'MWG'"),
    ("VEFAC là công ty con của ?", "VIC")
]

for question, expected_answer in test_pairs:
    response = query_engine.query(question)
    print(f"Question: {question}")
    print(f"Expected Answer: {expected_answer}")
    print(f"Actual Answer: {response.response}")
    print()

Question: lãi sau thuế của Chứng khoán LPBank ?
Expected Answer: Tăng 275% so với cùng kỳ năm trước
Actual Answer: Lãi sau thuế của Chứng khoán LPBank đạt 33 tỷ đồng trong quý 3/2024, tăng 275% so với cùng kỳ năm trước.

Question: thu gom cố phiếu MSN?
Expected Answer: Khối ngoại cũng giải ngân mua ròng 179 tỷ tại cổ phiếu MSN
Actual Answer: Cổ phiếu MSN đã thu hút dòng vốn khối ngoại với mức mua ròng 179 tỷ đồng trong tuần qua.

Question: Doanh thu Bia Sài Gòn - Quảng Ngãi (BSQ) trong BCTC quý 3/2024 ?
Expected Answer: Bia Sài Gòn - Quảng Ngãi (BSQ) ghi nhận doanh thu 418 tỷ đồng
Actual Answer: Doanh thu của Bia Sài Gòn - Quảng Ngãi (BSQ) trong BCTC quý 3/2024 ghi nhận là 418 tỷ đồng, giảm hơn 1% so với năm trước.

Question: Lực mua ròng các cổ phiếu nào ?
Expected Answer: ACV', 'YEG', 'TCB', 'TPB', 'MWG'
Actual Answer: Lực mua ròng ghi nhận tại các cổ phiếu STB, MSN, NTL, ACV, YEG, TCB, TPB, và MWG trong tuần qua.

Question: VEFAC là công ty con của ?
Expected Answer: VIC
Actual Answ

## Kết luận.

Hóa ra loại cơ bản nhất lại chính xác nhất :v